<h2>This notebook studies the <b>Team Orienteering </b> with Pick-up, Delivery Problem and Transfer (TOPDPT)</h2>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

from gurobipy import Model, quicksum, GRB
import numpy as np
from util import generate_node_cargo_size_change, read_pickle, group_cycle_truck, manual_stop
from pathlib import Path
from tvopdpt import tvopdpt_milp_gurobi
dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
num_ins = 10

In [3]:
def solve_pdotw_mip(ins,  # dict contains the data of pdpt instance,
                    path_, # file where all data of pdotw solutions are saved
                    optimize_pdotw_routes = True,
                    max_runtime = 100,
                    verbose = 0):  

    # load data from ins
    selected_truck = ins['truck']
    selected_cargo = ins['cargo']
    selected_node = ins['nodes']
    selected_edge = ins['edge_shortest']    
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']


    # edge_shortest, path_shortest = replace_edge_by_shortest_length_nx(nodes, edges)
    # single_truck_deviation = calculate_single_truck_deviation(truck, cargo, edge_shortest)
    
    start_time = time.time()
    print(f'========= START [PDOTW with truck ========= ')

    
    created_truck = selected_truck.copy()
    
    # nodes in the cluster
    # Note. cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time','departure_node', 'arrival_node']
    # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']
    

    if verbose >0:
        print(f'+++ Preprocess data to instantiate a PDOTW MIP')
    if verbose > 2:
        print(f'    [selected_cargo] size: {len(selected_cargo)}')
        for key, value in selected_cargo.items():
            print(f'        {key, value}')
        print(f'    [created_truck] size: {len(created_truck)}')
        for key, value in created_truck.items():
            print(f'       {key, value}')
    
    ### Need to update node_cargo_size_change 
    node_cargo_size_change = generate_node_cargo_size_change(selected_node, selected_cargo)

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, selected_truck = group_cycle_truck(created_truck)  
    
    if verbose > 2:
        print('    [created_truck_yCycle]', created_truck_yCycle)
        print('    [created_truck_nCycle]', created_truck_nCycle)


    if verbose >0:
        print(f'+++ Solve TVOPDPT MILP in Gurobi')
    ### use gurobi to solve the GROW origin PDPTW
    # Note. the pdotw_mip_gurobi function is desgined to take the same arguments as pdpt function
    # but some parameters
    gurobi_log_file = os.path.join(path_, f'_gurobi.log')


    obj_val, model_runtime, milp_sol = tvopdpt_milp_gurobi(constant, 
                                                           selected_cargo, 
                                                           selected_truck, 
                                                           created_truck_yCycle, 
                                                           created_truck_nCycle,
                                                           selected_node, 
                                                           selected_edge, 
                                                           node_cargo_size_change, 
                                                           max_runtime, 
                                                           gurobi_log_file, 
                                                           verbose = 1)

    x_sol, z_sol, y_sol, S_sol, D_sol, A_sol, Sb_sol, Db_sol, Ab_sol, u_sol, w_sol = milp_sol
    # Index k for truck is pre-defined
    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #z_sol: z^{kr}_{ij}, if truck k visit edge (i,j) with cargo r
    #u_sol: u^r_i, if cargo r is transfered at node i
    #y_sol: y^k_r, if parcel r is carried by truck k
    #S_sol: x^k_i, total size of cargos on truck k at node i
    #D_sol: D^k_i, depature time of truck k at node i
    #A_sol: A^k_i, arrival time of truck k at node i

    res = {'x_sol':  x_sol,
           'z_sol':  z_sol,
           'y_sol':  y_sol,
           'S_sol':  S_sol,
           'D_sol':  D_sol,
           'A_sol':  A_sol,
           'Sb_sol': Sb_sol,
           'Db_sol': Db_sol,
           'Ab_sol': Ab_sol,
           'u_sol':  u_sol,
           'w_sol':  w_sol
          }

    return res

In [4]:

def main():
    case_num, ins_idx, num_trucks = 1, 0, 2
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
    print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
    pdpt_ins = read_pickle(pdpt_ins_filename)


    path_ = os.path.join(dir_, 'toy/top_pdpt')
    Path(path_).mkdir(parents=True, exist_ok=True)
    res = solve_pdotw_mip(pdpt_ins,  # dict contains the data of pdpt instance,
                          path_, # file where all data of pdotw solutions are saved
                          optimize_pdotw_routes = True,
                          max_runtime = 100,
                          verbose = 0)

    res_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
    with open(res_filename, 'wb') as f:
        pickle.dump(res, f)

main()

===== START team orienteering pdpt
      /home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins1.pkl
========= START [PDOTW with truck ========= 
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15
Set parameter Heuristics to value 0.5
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/toy/top_pdpt/_gurobi.log"
9
+++ Gurobi MIP for TVOPDPT [Feasible] 


ValueError: too many values to unpack (expected 3)

In [ ]:
def postprocess_pdpt_solution(pdpt_ins, tvopdpt_res, verbose = 0):

    selected_cargo = pdpt_ins['cargo']
    selected_truck = pdpt_ins['truck']
    selected_node  = pdpt_ins['nodes']
    selected_edge  = pdpt_ins['edge_shortest']
    
    x_sol = tvopdpt_res['x_sol']
    y_sol = tvopdpt_res['y_sol']
    z_sol = tvopdpt_res['z_sol']
    u_sol = tvopdpt_res['u_sol']
    w_sol = tvopdpt_res['w_sol']

    truck_used = []         # list of trucks used in the solution
    
    # dictionary, for each truck_key, there is a list of cargo carried that was on this truck. E.g., {'T1": ['C1', 'C2', ...]}
    cargo_in_truck = {}   
    for truck_key in selected_truck.keys():
        cargo_in_truck[truck_key] = []

    # dictionary, for each cargo_key, there is a list of truck that carried this cargo. E.g., {'C1": ['T1', 'T2', ...]}
    trucks_per_cargo = {} 
    for cargo_key in selected_cargo.keys():
        trucks_per_cargo[cargo_key] = []

    cargo_delivered = []   # list of delivered cargo
    cargo_undelivered = [] # list of undelivered cargo


    truck_route = {}
    for truck_key in selected_truck.keys():
        truck_route[truck_key] = []
    cargo_route = {}
    for cargo_key in selected_cargo.keys():
        cargo_route[cargo_key] = []

    # dict, for each cargo_key, there is a nested list, each with [transfer_node, truck_from, truck_to]
    cargo_transfer = {}
    for cargo_key in selected_cargo.keys():
        cargo_transfer[cargo_key] = []

    # postprocess cargo_in_truck and truck_used
    for truck_key in selected_truck:
        truck_used_flag = 0
        # Generate cargo_in_truck
        for cargo_key in selected_cargo.keys():
            if y_sol[(truck_key, cargo_key)] == 1:
                if verbose > 0: 
                    print('    The cargo {} has been carried by truck {}'.format(cargo_key, truck_key))
                truck_used_flag += 1
                cargo_in_truck[truck_key].append(cargo_key)
        if truck_used_flag > 0:
            
            # put the truck_key into used trucks
            truck_used.append(truck_key)

    # postprocess truck_route
    for truck_key in selected_truck:
        source = selected_truck[truck_key][0] # starting from the origin node of each truck
        for node_ in selected_node:
            if node_ != source and x_sol[(source, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1

                if len(truck_route[truck_key]) == 0: # append source as the first node
                        truck_route[truck_key].append(source)
                truck_route[truck_key].append(node_)
                source = node_
                break
        while source != selected_truck[truck_key][1]: # terminate when reach the arival node of each truck
            for node_ in selected_node:
                if node_ != source and x_sol[(source, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1
                    truck_route[truck_key].append(node_)
                    source = node_
                    break

    # RECALL, cargo is a dictionary with the following format:
    # cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']

    for cargo_key, cargo_value in selected_cargo.items():
        dest = cargo_value[-1]
        if sum([z_sol[(node_, dest, truck_key, cargo_key)] for node_ in selected_node for truck_key in selected_truck.keys() if node_!= dest]) == 1:
            assert sum([y_sol[(truck_, cargo_key)] for truck_ in selected_truck]) == 1
            cargo_delivered.append(cargo_key)
            truck_list = [truck_key for truck_key in selected_truck.keys()  # list of truck_key
                                        if y_sol[(truck_key, cargo_key)]  == 1  ]# if cargo_key is carried by truck_key i.e., y^k_r == 1
            n_curr = cargo_value[3]
            for n_next in selected_node:
                for truck_key in truck_list:
                    if n_next!=n_curr and z_sol[(n_curr, n_next, truck_key, cargo_key)] == 1:
                        if len(cargo_route[cargo_key]) == 0: # append source as the first node
                            cargo_route[cargo_key].append((truck_key, n_curr))
                        cargo_route[cargo_key].append((truck_key, n_next))
                        n_curr = n_next
                        break
                else:
                    continue
                break
            while n_curr != cargo_value[-1]:
                for n_next in selected_node:
                    for truck_key in truck_list:
                        if n_next!=n_curr and z_sol[(n_curr, n_next, truck_key, cargo_key)] == 1:
                            cargo_route[cargo_key].append((truck_key, n_next))
                            n_curr = n_next
                            break
                    else:
                        continue
                    break
        else:
            print([ [truck_key, cargo_key, sum(z_sol[(node_, dest, truck_key, cargo_key)] for node_ in selected_node if node_!= dest) ] for truck_key in selected_truck.keys()])

            assert sum([y_sol[(truck_, cargo_key)] for truck_ in selected_truck]) == 0, [[(truck_, cargo_key), y_sol[(truck_, cargo_key)]] for truck_ in selected_truck]
            cargo_undelivered.append(cargo_key)
    truck_1, truck_2 = selected_truck.keys()
    
    for cargo_ in selected_cargo.keys():
        for node_ in selected_node:
            if u_sol[node_, cargo_key] == 1:
#                 print(sum([z_sol[(node_prev, node_, truck_1, cargo_)] for node_prev in selected_node]))
#                 print(sum([z_sol[(node_, node_next, truck_2, cargo_)] for node_next in selected_node]))
                assert sum([z_sol[(node_prev, node_, truck_1, cargo_)] for node_prev in selected_node]) - sum([z_sol[(node_, node_next, truck_2, cargo_)] for node_next in selected_node]) == 1
                cargo_transfer[(node_, cargo_key)] = [node_, truck_1, truck_2]
            if w_sol[node_, cargo_key] == 1:
                assert sum([z_sol[(node_prev, node_, truck_2, cargo_)] for node_prev in selected_node]) - sum([z_sol[(node_, node_next, truck_1, cargo_)] for node_next in selected_node]) == 1
                cargo_transfer[(node_, cargo_key)] = [node_, truck_2, truck_1]
                

    trucks_per_cargo = {cargo_key: [truck_key  for truck_key in selected_truck.keys() if y_sol[(truck_key, cargo_key)]==1 ] for cargo_key, cargo_value in selected_cargo.items()}


    processed_res = (truck_used, cargo_delivered, cargo_undelivered, trucks_per_cargo, cargo_in_truck, truck_route, cargo_route, cargo_transfer)


    return processed_res


def read_result():
    case_num, ins_idx, num_trucks = 1, 0, 2
    
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
    pdpt_ins = read_pickle(pdpt_ins_filename)
    
    print('=== summary of pdpt_ins')
    
    print('[', len(pdpt_ins['cargo'].keys()),'] cargos, [', len(pdpt_ins['truck'].keys()),'] trucks')
    res_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
    tvopdpt_res = read_pickle(res_filename, verbose = 0) 
    processed_res  = postprocess_pdpt_solution(pdpt_ins, tvopdpt_res, verbose = 0)
    
    truck_used, cargo_delivered, cargo_undelivered, trucks_per_cargo, cargo_in_truck, truck_route, cargo_route, cargo_transfer = processed_res

    print(f'+++ list of cargos sucessfully delivered \n{cargo_delivered}')
          
    print(cargo_undelivered)
    
read_result()

<h3>How to encode transfer in TOPDPT ?</h3>

Here we consider a simple case of only two trucks $k_1, k_2$.

On top of the single-vehicle Pick-up Delivery Orienteering Problem with Time-Window (PDOTW), we introduce two sets of new variables $z^{kr}_{ij}, u^r_i \in \{0,1\}$.

We add constraints to encode the following situations. 

$$\text{from Jason's code}\left\{\begin{aligned} 
&\sum_{i\in\mathcal{V}}u^r_i,\;  \forall <= 1 &&\text{each cargo } r \text{ can only be transfered at most once}\\
&u^r_i >= \sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ij} - \sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ji},\; \forall i\in\mathcal{V}, r, k && u^r_i = 1 \text{ IFF } \underbrace{\sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ij} = 1}_{\text{ truck }k \text{ arrives node } i \text{ with cargo }r } \text{ and } \underbrace{\sum_{j\in\mathcal{V}: j\neq i}z^{kr}_{ji}}_{\text{ truck }k \text{ leaves node } i \text{ without cargo }r }
\end{aligned}\right.$$


\begin{aligned}
&\text{Does not consider transfer}\\
&DT^k_i \ge AT^k_i + \underbrace{\text{fixed time} + \text{load pick-up cargos} + \text{unload delivery cargos}}_{\text{time that } k \text{ spends on node } i\text{, these terms are independent of cargo transfer}}, \; k\in\{k_1,k_2\}, i\in\mathcal{V}\\ \\
&\text{let } T^k = \text{fixed time} + \text{load pick-up cargos} + \text{unload delivery cargos} \\ \\
&\text{Consider transfer from }k_2 \text{ to } k_1\\
&\quad \text{1. if } k_2 \text{ finishes unloading transfer cargos before the arrival of } k_1, \\
&\quad \text{that is, } AT^{k_2}_i + \text{unload }r \le AT^{k_1}_i\\
&\quad DT^{k_1}_i \ge AT^{k_1}_i + \underbrace{\text{load }r}_{\forall u^r_i=1} + T^{k_1}\\
&\quad\quad\quad\quad - M\Big(2 - u^r_i - \sum_{j\in\mathcal{j}:\:j\neq i} z^{k_1r}_{ij}\Big) , \; \forall i\in\mathcal{V}, r
\\ \\
&\quad \text{2. if } k_2 \text{ after } k_1 \text{ and before } k_1 \text{ finished everything} \\
&\quad \text{that is, } AT^{k_1}_i \le AT^{k_2}_i \le AT^{k_1}_i + T^{k_1}\\
&\quad DT^{k_1}_i \ge AT^{k_1}_i + \underbrace{\text{unload and load}r}_{\forall u^r_i=1}   + T\\
&\quad\quad\quad\quad - M\Big(2 - u^r_i - \sum_{j\in\mathcal{j}:\:j\neq i} z^{k_1r}_{ij}\Big) , \; \forall i\in\mathcal{V}, r \\ \\
&\quad \text{3. if } k_2 \text{ arrives after } k_1 \text{ finished everything} \\
&\quad \text{that is, } AT^{k_2}_i \ge AT^{k_1}_i + T^{k_1}\\
&\quad DT^{k_1}_i \ge AT^{k_2}_i + \underbrace{\text{unload and load}r}_{\forall u^r_i=1}   + T\\
&\quad\quad\quad\quad - M\Big(2 - u^r_i - \sum_{j\in\mathcal{j}:\:j\neq i} z^{k_1r}_{ij}\Big) , \; \forall i\in\mathcal{V}, r
\end{aligned}

the last two constraints are controled by Big-M, it is active IFF $\underbrace{u^r_r = 1}_{ r \text{ is transfered at } i}$ and $\underbrace{\sum_{j\in\mathcal{j}:\:j\neq i} z^{kr}_{ij}=1}_{r \text{ is not carried by }k_1 \text{ when arrives at } i}$

In [ ]:
print(*[[]]*11)